# Генерация фич и текстовый препроцессинг

## Общее описание
Ноутбук выполняет комплексную подготовку данных для задачи машинного обучения:
- Генерация новых признаков
- Текстовый препроцессинг
- Оптимизация типов данных
- Очистка от нерелевантных признаков

## Результирующие признаки

### Числовые признаки
| Признак | Описание | Тип данных |
|---------|----------|------------|
| `price_diff_log` | Логарифмированная разница цен (log1p) | float32 |
| `price_diff_is_outlier` | Флаг выбросов (метод IQR) | bool |
| `param1` | Совпадение значений параметра | int8 |

### Текстовые признаки
| Признак | Описание | Тип данных |
|---------|----------|------------|
| `base_text` | Конкатенация: base_title + base_description | object |
| `cand_text` | Конкатенация: cand_title + cand_description | object |

### Удаленные признаки
- Исходные текстовые поля (title/description)
- Ценовые показатели (base_price, cand_price)
- Категориальные признаки
- Параметры объявлений
- Географические признаки
- Мета-информация (images, json_params)

In [ ]:
import pandas as pd
import numpy as np

import re

import warnings
warnings.filterwarnings('ignore')

# Загружаем отдельно test и train часть для обработки

In [2]:
train = pd.read_parquet('../data/train.parquet')

In [2]:
test = pd.read_parquet('../data/test.parquet')

In [61]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500000 entries, 0 to 499999
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   base_item_id           500000 non-null  object 
 1   cand_item_id           500000 non-null  object 
 2   base_title             500000 non-null  object 
 3   cand_title             500000 non-null  object 
 4   base_description       500000 non-null  object 
 5   cand_description       500000 non-null  object 
 6   base_category_name     500000 non-null  object 
 7   cand_category_name     500000 non-null  object 
 8   base_subcategory_name  500000 non-null  object 
 9   cand_subcategory_name  500000 non-null  object 
 10  base_param1            481725 non-null  object 
 11  cand_param1            481729 non-null  object 
 12  base_param2            365761 non-null  object 
 13  cand_param2            365581 non-null  object 
 14  base_price             500000 non-null  f

# Function

In [3]:
def some_diff(data: pd.DataFrame, 
              col1: str, 
              col2: str,
              name: str) -> pd.DataFrame:
    """Считает разность значений в двух столбцах
    :param data: датасет
    :param col1: название столбца 1
    :param col2: название столбца 2
    :param name: название нового столбца
    :return: датасет с новым столбцом
    """
    data[name] = abs(data[col1] - data[col2])
    
    return data

In [4]:
def price(data: pd.DataFrame) -> pd.DataFrame:
    """Преобразует признак цены"""
    data = some_diff(data, 'base_price', 'cand_price', 'price_diff')
    
    # прологарифмируем фичу, так как присутствуют сильные выбросы (log(1 + x))
    data['price_diff_log'] = np.log1p(data['price_diff'])
    
    # создадим флаг выброса как фичу
    Q1 = data['price_diff_log'].quantile(0.25)
    Q3 = data['price_diff_log'].quantile(0.75)
    IQR = Q3 - Q1

    # нижняя и верхняя границы для определения выбросов
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # флаг выброса
    data['price_diff_is_outlier'] = (data['price_diff_log'] < lower_bound) | (
        data['price_diff_log'] > upper_bound)
    
    # преобразуем в оптимальных тип данных
    data['price_diff_log'] = data['price_diff_log'].astype('float32')
    
    return data

In [5]:
def combine_columns(data: pd.DataFrame) -> pd.DataFrame:
    """Соединяет текстовые фичи в новую и удаляет старые столбцы"""
    data['base_text'] = data['base_title'].astype(
        str) + '. ' + data['base_description'].astype(str)
    data['cand_text'] = data['cand_title'].astype(
        str) + '. ' + data['cand_description'].astype(str)
    
    return data

In [6]:
def cleaned_text(text: str) -> str:
    """Простая очистка текста"""
    # преобразуем text в строку, если это не строка
    text = str(text) if text is not None else ''
    text = text.lower()
    text = re.sub(r'[^а-яёa-z0-9\s.,*!?:-]', '', text) # удаление лишних символов (кроме пунктуации)
    text = re.sub(r'\s+', ' ', text).strip() # удаление лишних пробелов
    text = re.sub(r'^[^\w]+', '', text) # удаление пунктуации в начале строки
    text = text.strip(' .') # убирает лишние пробелы и точки в начале и в конце строки
    
    return text

# Preprocessing

In [7]:
def preprocessing(data: pd.DataFrame) -> pd.DataFrame:
    """
    Препроцессинг датасета
    """
    # получаем признаки из цены
    data = price(data)

    # получаем фичу param1
    data['param1'] = (data['base_param1'].astype(str) ==
                      data['cand_param1'].astype(str)).astype(int)

    # получаем текстовые признаки
    data = combine_columns(data)

    # удаляем столбцы датасета
    data.drop([
        'base_title', 'base_description', 'cand_title', 'cand_description',
        'base_price', 'cand_price', 'price_diff', 'base_category_name',
        'cand_category_name', 'base_subcategory_name', 'cand_subcategory_name',
        'base_param1', 'cand_param1', 'base_param2', 'cand_param2',
        'base_count_images', 'cand_count_images', 'base_json_params',
        'cand_json_params', 'is_same_location', 'is_same_region',
        'base_count_images', 'cand_count_images'
    ],
              axis=1,
              inplace=True)

    # преобразовываем данные в оптимальный тип
    to_int32 = ['group_id']
    to_int8 = ['is_double', 'param1', 'count_images_diff']

    for col in data.columns:
        if data[col].dtype == 'int64' and col in to_int32:
            data[col] = data[col].astype('int32')
        elif data[col].dtype == 'int64' and col in to_int8:
            data[col] = data[col].astype('int8')

    # чистка текста
    for i in data.columns:
        if data[i].dtype == 'object':
            data[i] = data[i].apply(cleaned_text)
            data[i] = data[i].astype('object')
        elif data[i].dtype == 'category':
            data[i] = data[i].apply(cleaned_text)
            data[i] = data[i].astype('category')

    return data

## Test

In [8]:
df_test = preprocessing(test)
df_test.head()

,base_item_id,cand_item_id,base_title_image,cand_title_image,price_diff_log,price_diff_is_outlier,param1,base_text,cand_text
0,c66017e8712f80266bbed8b68285ee1ed8bead00ebb450...,a4b4bc67f198e9ceda2e5d551f2d323cd4be6bc7fe3fb7...,cfc47375008a976fda632ffdd07d5d5218adc8f9b41a13...,4e44b2536cc28e597c824f57b881413482d7a379eb7dfe...,10.913287,False,1,смapт-чacы huawei watch d2. абcoлютнo нoвые чa...,смapт чacы garmin fenix 7 pro sapphire solar. ...
1,c9a60f42a86c7d27df6c73be26bfef58efe58a71431955...,f6dee23c99b365055d2365ccb7b60d52369a11e753ebf6...,9efcb5c59529a7a95f393842ca482d14f989e478a602bf...,e8693c9284da135b01b089cb96b1ca61c09b5ac5dc07d2...,7.264730,False,1,"диcк 7x175x114,3 et45 d67,1 rebel кc913 хaй вэ...","диcк 6,5x185x108 et33 d60,1 бoмбей кc1075 хaй ..."
2,81912ec6d2b220e0ed65413588acea910fe9b71229d281...,fdd213efde102f6dd1dce6830b323e9b8aa1484607bd91...,434f5a021df1726564bb6176ca0fc2a5cba252a0af036a...,246771448a8a66f2530a349edc9093412fc1dd0a665107...,6.311735,False,1,книги для пoдpocткoв и взpocлых. 1 книгa. aвит...,лучшие книги. идеaльнo в пoдapoк. любую пoзици...
3,0ca0d9733cf8b7d9aa097270c3e06bcd69a9061573d254...,4f232dbcb2884c52114df6c2927cdeda91135ed85278f4...,e0662ceaf20ca2eaa48528688e8888bad68bf207c5a22d...,835ea8cdb00c4a77ef4d3dba5e0bf8cc4334440194f19d...,10.308986,False,1,вocпитaтель в чacтный детcкий caд. к нaм в кoм...,вocпитaтель в чacтный детcкий caд. вaкaнcия: в...
4,8d0ec90fb0e1ff6b71c7c77ad113282b27859d200406a0...,819b0ef141e76a2fbe6688fddf86dcbbe355c73c730e60...,7a2e5d0dd1a293feb77bf8c17659df6f3e9217059c6e8f...,74453401ed7ba7e30ed2113434e3f6c4e710ec47de4b29...,8.537192,False,1,пеpфopaтop. гapaнтия пo чеку 6 меcяцев! вcе дo...,плиткopез вoдянoй. без зaлoгa нужнo фoтo пacпo...


In [9]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500000 entries, 0 to 499999
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   base_item_id           500000 non-null  object 
 1   cand_item_id           500000 non-null  object 
 2   base_title_image       500000 non-null  object 
 3   cand_title_image       500000 non-null  object 
 4   price_diff_log         500000 non-null  float32
 5   price_diff_is_outlier  500000 non-null  bool   
 6   param1                 500000 non-null  int8   
 7   base_text              500000 non-null  object 
 8   cand_text              500000 non-null  object 
dtypes: bool(1), float32(1), int8(1), object(6)
memory usage: 29.6+ MB


In [10]:
# сохраняем предобработанный датасет
#df_test.to_parquet('../data/df_test.parquet', index=False)

## Train

In [8]:
df_train = preprocessing(train)

df_train.drop(['action_date', 'group_id'], axis=1, inplace=True)
df_train.rename(columns={'is_double': 'target'}, inplace=True) 
df_train['target'] = df_train.pop('target')  # переносит target в последнюю колонку

df_train.head()

,base_item_id,cand_item_id,base_title_image,cand_title_image,price_diff_log,price_diff_is_outlier,param1,base_text,cand_text,target
0,13ade32c3e614d939faad4ab68350bc52ce8848b7a64bd...,087e7f3dbec9326532f9fc784b68de294cb2d905d33bdf...,40c72f08e0bb10b55e0605781481df2b5557b094aee695...,ebc7537d69a1c8c1a6e7ea3c5b27ab4d4a360e6032d158...,7.378384,False,1,"зимние ботинки ecco. ботинки экко,униcекc,зимн...",кигуpуммии мышкa inextenso,0
1,13ade32c3e614d939faad4ab68350bc52ce8848b7a64bd...,5d81d4230671ed22e40ab9e05bb63fef5ad6766454714a...,40c72f08e0bb10b55e0605781481df2b5557b094aee695...,8f8e254c919ecb28a3424fde4cd1aeca75043584ddd02c...,7.346655,False,1,"зимние ботинки ecco. ботинки экко,униcекc,зимн...","штaны для девочки zara. штaны новые,ноcили пap...",0
2,13ade32c3e614d939faad4ab68350bc52ce8848b7a64bd...,eff6d2ef2c44dc7361d389d3a9ce243e6e3079675c0b27...,40c72f08e0bb10b55e0605781481df2b5557b094aee695...,a92f75d133c370f8b5d135d29144a69e1179d522e06d78...,7.467942,False,1,"зимние ботинки ecco. ботинки экко,униcекc,зимн...","рубaшкa acoola 152. новaя,не ноcили",0
3,d0b78018657dff01508954bb58d4f03f1ddf11525d8d26...,13ade32c3e614d939faad4ab68350bc52ce8848b7a64bd...,181549e281126b799e54980db0b194918479e0db9be2ab...,40c72f08e0bb10b55e0605781481df2b5557b094aee695...,7.972811,False,1,куpткa зимняя и ветpовкa. зимнюю куpтку ноcили...,"зимние ботинки ecco. ботинки экко,униcекc,зимн...",0
4,d0b78018657dff01508954bb58d4f03f1ddf11525d8d26...,b960b579cd9b5aebc6ac73d5042ba13ae8747490cc8a59...,181549e281126b799e54980db0b194918479e0db9be2ab...,eda895d18bd2d5bba2b277475667835d7ab7f9186ba32e...,8.243019,False,1,куpткa зимняя и ветpовкa. зимнюю куpтку ноcили...,плaщ детcкий next. .пеpед отпpaвкой отпapю,0


In [9]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1879555 entries, 0 to 1879554
Data columns (total 10 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   base_item_id           object 
 1   cand_item_id           object 
 2   base_title_image       object 
 3   cand_title_image       object 
 4   price_diff_log         float32
 5   price_diff_is_outlier  bool   
 6   param1                 int8   
 7   base_text              object 
 8   cand_text              object 
 9   target                 int8   
dtypes: bool(1), float32(1), int8(2), object(6)
memory usage: 112.9+ MB


In [10]:
# сохраняем предобработанный датасет
#df_train.to_parquet('../data/df_train.parquet', index=False)

**Выводы:**
После просмотра и анализа тестовых данных, была немного изменена предобработка тренировочных данных. А именно из-за пропусков в столбцах количества изображений в тестовых данных, на трейне принято решение ее не использовать (по графикам в EDA также видно, что она несет много шума), также исходя из этого в трейне не были удалены строки с пропусками. На тесте отсутствует столбец action_date, следовательно фичи на его основе удалены из трейна. 

 
 Следующие шаги:
 1. На основе полученных эмбеддингов картинок и текстов, соответственно, посчитать их косинусную близость, разность и произведение.
 2. Из получившихся результатов разности и произведения соответствующих эмбеддингов, представить их в более низкоразмерном пространстве (например размерами 64 или 32 с помощью PCA или TNSE).
 3. Составить на основе всего вышеизложенного матрицу объект-признаки и обучить катбуст на задачу классификации.